## Big data
Hipótese a ser provada: Cidades que possuem um PIB per capita maior possuem maiores emissões de GEE.
- análise do aumento das emissões conforme variação do PIB per capita
- análise das cidades com maior crescimento nas emissões de GEE
- análise dos setores com maior crescimento nas emissões de GEE
- análise dos setores com maior redução nas emissões de GEE --> relação com crescimmento das cidades

In [ ]:
#%pip install duckdb

   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ----- ---------------------------------- 1.8/12.3 MB 11.2 MB/s eta 0:00:01
   ----------------- ---------------------- 5.5/12.3 MB 13.4 MB/s eta 0:00:01
   --------------------------- ------------ 8.4/12.3 MB 13.7 MB/s eta 0:00:01
   -------------------------------------- - 11.8/12.3 MB 14.2 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 13.6 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# Célula 1
import duckdb
import pandas as pd
from pathlib import Path

PARQUET_PATH = Path(r"C:\Users\Vitoria Oliveira\Desktop\bigdata_dp\aps\emissoes_gee\emissoes_por_decada_streaming.parquet")
df = pd.read_parquet(PARQUET_PATH)
conn = duckdb.connect(database=':memory:')
conn.register('emissoes', df)

In [16]:
# ve quais colunas eu tenho
conn.execute("PRAGMA table_info('emissoes');").df()

,cid,name,type,notnull,dflt_value,pk
0,0,decada,VARCHAR,False,None,False
1,1,Setor_Agrupado,VARCHAR,False,None,False
2,2,Cidade,VARCHAR,False,None,False
3,3,Gás,VARCHAR,False,None,False
4,4,Bioma,VARCHAR,False,None,False
5,5,emissao_t,DOUBLE,False,None,False


In [ ]:
# ve quanto de emissão total por década
conn.execute("""
SELECT 
    decada,
    SUM(emissao_t) as emissao_total
FROM emissoes
GROUP BY decada
ORDER BY decada
""").df()

,decada,emissao_total
0,1970s,1.017847e+10
1,1980s,1.151769e+10
2,1990s,7.702860e+10
3,2000s,6.755515e+10
4,2010s,3.115889e+10
5,2020s,1.738720e+10


In [ ]:
# quanto cada setor emitiu
conn.execute("""
SELECT 
    Setor_Agrupado,
    SUM(emissao_t) as emissao_total,
    ROUND(SUM(emissao_t) * 100.0 / (SELECT SUM(emissao_t) FROM emissoes), 2) as percentual
FROM emissoes
GROUP BY Setor_Agrupado
ORDER BY emissao_total DESC
LIMIT 10
""").df()

,Setor_Agrupado,emissao_total,percentual
0,Mudança de Uso do Solo,1.318094e+11,61.36
1,Energia e Combustíveis,4.041248e+10,18.81
2,Agropecuária,3.339637e+10,15.55
3,Processos Industriais,6.670353e+09,3.11
4,Resíduos,2.537423e+09,1.18


In [34]:
# ver o quanto cada setor emitiu em cada decada
conn.execute("""
SELECT 
    decada,
    Setor_Agrupado,
    SUM(emissao_t) as emissao_total,
    ROUND(SUM(emissao_t) * 100.0 / SUM(SUM(emissao_t)) OVER (PARTITION BY decada), 2) as percentual_na_decada
FROM emissoes
GROUP BY Setor_Agrupado, decada
ORDER BY Setor_Agrupado, decada, emissao_total DESC
""").df()

,decada,Setor_Agrupado,emissao_total,percentual_na_decada
0,1970s,Agropecuária,3.726920e+09,36.62
1,1980s,Agropecuária,5.004822e+09,43.45
2,1990s,Agropecuária,7.757837e+09,10.07
3,2000s,Agropecuária,7.564222e+09,11.20
4,2010s,Agropecuária,6.592809e+09,21.16
5,2020s,Agropecuária,2.749762e+09,15.81
6,1970s,Energia e Combustíveis,5.640231e+09,55.41
7,1980s,Energia e Combustíveis,5.457033e+09,47.38
8,1990s,Energia e Combustíveis,6.431295e+09,8.35
9,2000s,Energia e Combustíveis,8.196754e+09,12.13


In [ ]:
# ver o quanto cada setor emitiu em cada decada, ordenado pela decada e emissao total
conn.execute("""
SELECT 
    decada,
    Setor_Agrupado,
    SUM(emissao_t) as emissao_total
FROM emissoes
GROUP BY decada, Setor_Agrupado
ORDER BY decada, emissao_total DESC
""").df()

,decada,Setor_Agrupado,emissao_total
0,1970s,Energia e Combustíveis,5.640231e+09
1,1970s,Agropecuária,3.726920e+09
2,1970s,Processos Industriais,6.833153e+08
3,1970s,Resíduos,1.280063e+08
4,1970s,Mudança de Uso do Solo,0.000000e+00
5,1980s,Energia e Combustíveis,5.457033e+09
6,1980s,Agropecuária,5.004822e+09
7,1980s,Processos Industriais,8.347400e+08
8,1980s,Resíduos,2.210964e+08
9,1980s,Mudança de Uso do Solo,0.000000e+00


In [45]:
# variação por setor
conn.execute("""
WITH primeira AS (
    SELECT MIN(decada) as dec FROM emissoes
),
ultima AS (
    SELECT MAX(decada) as dec FROM emissoes
)
SELECT 
    Setor_Agrupado,
    ROUND(
        (SUM(CASE WHEN e.decada = u.dec THEN emissao_t ELSE 0 END) - 
         SUM(CASE WHEN e.decada = p.dec THEN emissao_t ELSE 0 END)) * 100.0 /
        NULLIF(SUM(CASE WHEN e.decada = p.dec THEN emissao_t ELSE 0 END), 0),
        2
    ) as variacao_percentual
FROM emissoes e
CROSS JOIN primeira p
CROSS JOIN ultima u
GROUP BY Setor_Agrupado
ORDER BY variacao_percentual DESC
""").df()

,Setor_Agrupado,variacao_percentual
0,Resíduos,195.51
1,Processos Industriais,12.24
2,Energia e Combustíveis,-26.15
3,Agropecuária,-26.22
4,Mudança de Uso do Solo,NaN


In [48]:
# evolução das emissões totais por década
conn.execute("""
SELECT 
    decada,
    SUM(emissao_t) as emissao_total,
FROM emissoes
GROUP BY decada
ORDER BY decada
""").df()

,decada,emissao_total
0,1970s,1.017847e+10
1,1980s,1.151769e+10
2,1990s,7.702860e+10
3,2000s,6.755515e+10
4,2010s,3.115889e+10
5,2020s,1.738720e+10


In [46]:
# variação por década
conn.execute("""
WITH primeira AS ( 
    SELECT MIN(decada) as dec FROM emissoes
),
ultima AS (
    SELECT MAX(decada) as dec FROM emissoes
)
SELECT 
    decada,
    ROUND(
        (SUM(CASE WHEN e.decada = u.dec THEN emissao_t ELSE 0 END) - 
         SUM(CASE WHEN e.decada = p.dec THEN emissao_t ELSE 0 END)) * 100.0 /
        NULLIF(SUM(CASE WHEN e.decada = p.dec THEN emissao_t ELSE 0 END), 0),
        2
    ) as variacao_percentual
FROM emissoes e
CROSS JOIN primeira p
CROSS JOIN ultima u
GROUP BY decada
ORDER BY variacao_percentual DESC
""").df()

,decada,variacao_percentual
0,1970s,-100.0
1,2020s,NaN
2,2000s,NaN
3,2010s,NaN
4,1990s,NaN
5,1980s,NaN


In [26]:
# ve quais as 20 cidades que mais emitiram no total
conn.execute("""
SELECT 
    Cidade,
    SUM(emissao_t) as emissao_total,
FROM emissoes
GROUP BY Cidade
ORDER BY emissao_total DESC
LIMIT 20
""").df()

,Cidade,emissao_total
0,NA (NA),2.030709e+10
1,NA (SP),5.590600e+09
2,São Félix do Xingu (PA),4.911190e+09
3,Juara (MT),2.834464e+09
4,Colniza (MT),2.200245e+09
5,Novo Repartimento (PA),2.140946e+09
6,Cumaru do Norte (PA),2.131269e+09
7,Marabá (PA),2.130581e+09
8,Vila Bela da Santíssima Trindade (MT),2.074550e+09
9,Novo Progresso (PA),1.884832e+09


In [52]:
# municípios com maior crescimento
conn.execute("""
WITH primeira AS (SELECT MIN(decada) as dec FROM emissoes),
     ultima AS (SELECT MAX(decada) as dec FROM emissoes)
SELECT 
    Cidade,
    SUM(CASE WHEN e.decada = u.dec THEN emissao_t ELSE 0 END) - 
    SUM(CASE WHEN e.decada = p.dec THEN emissao_t ELSE 0 END) as variacao
FROM emissoes e
CROSS JOIN primeira p
CROSS JOIN ultima u
WHERE Cidade IS NOT NULL
GROUP BY Cidade
ORDER BY variacao DESC
LIMIT 10
""").df()

,Cidade,variacao
0,São Félix do Xingu (PA),7.536726e+08
1,Altamira (PA),4.497651e+08
2,Colniza (MT),4.119167e+08
3,Novo Progresso (PA),3.275525e+08
4,Lábrea (AM),3.154434e+08
5,Pacajá (PA),2.974676e+08
6,Apuí (AM),2.760896e+08
7,Portel (PA),2.749310e+08
8,Porto Velho (RO),2.373899e+08
9,Aripuanã (MT),2.340464e+08


In [ ]:
# emissões por bioma
conn.execute("""
SELECT 
    Bioma,
    SUM(emissao_t) as emissao_total,
    ROUND(SUM(emissao_t) * 100.0 / (SELECT SUM(emissao_t) FROM emissoes WHERE Bioma IS NOT NULL), 2) as percentual
FROM emissoes
WHERE Bioma IS NOT NULL
GROUP BY Bioma
ORDER BY emissao_total DESC
""").df()

,Bioma,emissao_total,percentual
0,Amazônia,1.206742e+11,63.95
1,Cerrado,3.822170e+10,20.25
2,Mata Atlântica,1.886251e+10,10.00
3,Pampa,7.241357e+09,3.84
4,Caatinga,2.626739e+09,1.39
5,Pantanal,1.089084e+09,0.58


In [ ]:
# tipo de gás
conn.execute("""
SELECT 
    "Gás",
    SUM(emissao_t) as emissao_total,
    ROUND(AVG(emissao_t), 2) as emissao_media,
FROM emissoes
WHERE "Gás" IS NOT NULL
GROUP BY "Gás"
ORDER BY emissao_total DESC
""").df()

,Gás,emissao_total,emissao_media
0,CO2e (t) GWP-AR5,4.758521e+10,662912.80
1,CO2e (t) GTP-AR5,3.662065e+10,510164.82
2,CO2e (t) GWP-AR6,3.320163e+10,939704.22
3,CO2e (t) GTP-AR6,2.720285e+10,769920.94
4,CO2 (t),2.348044e+10,625061.59
5,CO2e (t) GWP-AR2,2.169455e+10,570308.82
6,CO2e (t) GTP-AR2,1.780385e+10,468029.60
7,CO2e (t) GWP-AR4,4.587989e+09,218673.50
8,CO2e (t) GTP-AR4,2.171197e+09,103483.96
9,CH4 (t),2.372546e+08,7690.34
